In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time

In [2]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


In [3]:
root_address = 'https://classroom.udacity.com/nanodegrees/nd101/dashboard/overview'
# root_address = 'https://www.udacity.com'

In [ ]:
#setup option for chrome profile
chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.implicitly_wait(60)

driver.get(root_address)

wait = WebDriverWait(driver, 30)

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [ ]:
email = driver.find_element_by_xpath("//input[@data-testid='signin-email']")
email

In [ ]:
def sync_get_element_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element_by_xpath(xpath)

In [ ]:
def sync_get_elements_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_elements_by_xpath(xpath)

In [ ]:
def sign_in():
    email = sync_get_element_by_xpath("//input[@data-testid='signin-email']")
    password = sync_get_element_by_xpath("//input[@data-testid='signin-password']")

    email.send_keys("huohsien@gmail.com")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb")))
    sign_in_btn = driver.find_element_by_class_name("button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb") 
    sign_in_btn.click()

In [ ]:
def check_lesson_complete(item)->bool:
    e1 = item.find_element_by_xpath("./div/div")
    c = e1.get_attribute('class')
    if c == 'index--lesson-card--mwX1V index--card--3DZMr shared--card--3X88h':
        return True
    else:
        return False

In [ ]:
def expand_all_complete_lessons():
    #click open the ones that were marked Complete
    lessons = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    for lesson in lessons:
        if check_lesson_complete(lesson):
            lesson.click()
    element = driver.find_element_by_xpath("//*[h3='lesson 1']")
    driver.execute_script("return arguments[0].scrollIntoView(true);", element)
    print("scroll to the top")
#     time.sleep(1)

In [ ]:
if check_exists_by_xpath("//div[div='Sign in to your account']"):
    sign_in()

In [ ]:
# Get topic titles
topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
num_topics = len(topic_elms)

for idx_topics in range(num_topics):
    # iterate each topic
    #
    print("Topic: ", ''.join(topic_elms[idx_topics].text.split('\n')[0:2]))
    link_topic = topic_elms[idx_topics].find_element_by_xpath("./a")
    link_topic.click()     # click into a topic
    # In the lesson list page
    #
    expand_all_complete_lessons()
    
    #iterate each lesson
    #
    lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    num_lessons = len(lesson_elms)
    print("num_lessons= ", num_lessons)
    for idx_lesson in range(num_lessons):
        # iterate each lesson
        #
        lesson_elm = lesson_elms[idx_lesson]
        print(' - '.join(lesson_elm.text.split('\n')[1:3]))
        link_lesson = lesson_elm.find_element_by_xpath("//*[span='View Lesson']")
        link_lesson.click()  # click into a lesson
        
        print("go back to lesson list")
        driver.back()
        expand_all_complete_lessons()
        lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
        
    print("go back to topic list")
    driver.back()
    topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
    

In [ ]:
# link_topic.click() 

In [ ]:
# expand_all_complete_lessons()



In [ ]:
# element = driver.find_element_by_xpath("//*[h3='lesson 1']")
# driver.execute_script("return arguments[0].scrollIntoView(true);", element)